# cuGraph Cheat Sheets sample code

(c) 2020 NVIDIA, Blazing SQL

Distributed under Apache License 2.0

### Imports

In [1]:
import cudf
import cugraph
import json 
import pandas as pd

### Create graph dataframes

We use the Star Wars social network graph prepared by Evelina Gabasova: Gabasova, E. (2016). Star Wars social network. DOI: https://doi.org/10.5281/zenodo.1411479.

In [2]:
interactions = json.loads(open('data/starwars-full-interactions.json', 'r').read())

edges = cudf.from_pandas(pd.DataFrame(interactions['links']))
nodes = cudf.from_pandas(pd.DataFrame(interactions['nodes'])).drop('colour', axis=1).reset_index()

g = cugraph.Graph()
g.from_cudf_edgelist(
    edges
    , source='source'
    , destination='target'
    , edge_attr='value'
    , renumber=True
)

---

# Graph link analysis

---

#### cugraph.link_analysis.hits.hits()

In [3]:
cugraph.hits(g, max_iter=50)

,hubs,authorities,vertex
0,0.029666,0.029666,62
1,0.017394,0.017394,69
2,0.029488,0.029488,73
3,0.014282,0.014282,75
4,0.033828,0.033828,77
...,...,...,...
104,0.018747,0.018747,92
105,0.029854,0.029854,22
106,0.071162,0.071162,85
107,0.042893,0.042893,88


#### cugraph.link_analysis.pagerank.pagerank()

In [4]:
cugraph.pagerank(g, alpha = 0.85, max_iter = 500, tol = 1.0e-05)

,pagerank,vertex
0,0.003203,62
1,0.004878,69
2,0.011510,73
3,0.003540,75
4,0.004308,77
...,...,...
104,0.014623,92
105,0.005646,22
106,0.010707,85
107,0.005247,88


---

# Graph link prediction

---

#### cugraph.link_prediction.jaccard.jaccard()

In [6]:
pairs = g.get_two_hop_neighbors()
cugraph.jaccard(g, pairs)

,jaccard_coeff,source,destination
0,0.119048,17,47
1,0.119048,17,52
2,0.085106,17,57
3,0.024390,17,59
4,0.108108,23,21
...,...,...,...
4845,0.166667,87,89
4846,0.074074,87,0
4847,0.058824,87,14
4848,0.090909,87,45


#### cugraph.link_prediction.jaccard.jaccard_coefficient()

In [9]:
cugraph.jaccard_coefficient(g)

,jaccard_coeff,source,destination
0,0.166667,17,10
1,0.142857,17,15
2,0.095238,17,16
3,0.166667,17,20
4,0.095238,17,24
...,...,...,...
791,0.260870,8,38
792,0.086957,8,39
793,0.304348,8,10
794,0.130435,8,28


In [10]:
cugraph.jaccard_coefficient(g, pairs)

,jaccard_coeff,source,destination
0,0.600000,107,105
1,0.500000,108,107
2,0.266667,108,91
3,0.500000,108,109
4,0.083333,108,103
...,...,...,...
791,0.138889,13,4
792,0.185185,13,0
793,0.147059,13,14
794,0.500000,13,9


#### cugraph.link_prediction.wjaccard.jaccard_w()

In [13]:
cugraph.jaccard_w(g, weights=edges['value'])

,jaccard_coeff,source,destination
0,0.118421,63,8
1,0.222222,63,4
2,0.288732,63,70
3,0.325301,63,45
4,0.282443,63,67
...,...,...,...
791,0.508197,8,31
792,0.352381,30,0
793,0.452055,30,29
794,0.227273,30,32


In [14]:
cugraph.jaccard_w(g, weights=edges['value'], vertex_pair=pairs)

,jaccard_coeff,source,destination
0,0.176923,94,48
1,0.289855,94,93
2,0.681818,94,104
3,0.012987,94,23
4,0.432432,85,104
...,...,...,...
4845,0.227907,21,85
4846,0.121359,21,88
4847,0.052863,21,107
4848,0.056769,21,108


#### cugraph.link_prediction.overlap.overlap()

In [15]:
cugraph.overlap(g)

,overlap_coeff,source,destination
0,0.750000,43,4
1,0.500000,43,45
2,0.500000,43,44
3,0.500000,43,46
4,0.833333,51,17
...,...,...,...
791,0.857143,8,38
792,0.666667,8,39
793,0.875000,8,10
794,0.750000,8,28


In [16]:
cugraph.overlap(g, vertex_pair=pairs)

,overlap_coeff,source,destination
0,0.500000,84,63
1,1.000000,84,45
2,0.500000,84,67
3,0.500000,84,64
4,0.250000,13,34
...,...,...,...
4845,0.333333,55,5
4846,0.333333,55,27
4847,0.666667,55,37
4848,0.333333,55,41


#### cugraph.link_prediction.overlap.overlap_coefficient()

In [17]:
cugraph.overlap_coefficient(g)

,overlap_coeff,source,destination
0,0.266667,63,8
1,0.466667,63,4
2,0.400000,63,70
3,0.666667,63,45
4,0.533333,63,67
...,...,...,...
791,0.857143,8,38
792,0.666667,8,39
793,0.875000,8,10
794,0.750000,8,28


#### cugraph.link_prediction.woverlap.overlap_w()

In [18]:
cugraph.overlap_w(g, weights=edges['value'])

,overlap_coeff,source,destination
0,0.909091,43,4
1,0.818182,43,45
2,0.500000,43,44
3,0.333333,43,46
4,0.900000,51,17
...,...,...,...
791,0.972973,8,38
792,0.750000,8,39
793,0.916667,8,10
794,0.966667,8,28


In [19]:
cugraph.overlap_w(g, weights=edges['value'], vertex_pair=pairs)

,overlap_coeff,source,destination
0,0.674699,4,11
1,0.500000,4,33
2,0.363636,4,70
3,0.575342,4,63
4,0.600000,89,90
...,...,...,...
4845,0.111111,22,21
4846,0.888889,22,8
4847,0.333333,22,11
4848,0.111111,22,33
